In [13]:
import raster_geometry as rg
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import rotate
import pickle
import sys
from skimage.transform import rescale, resize, downscale_local_mean
sys.path.insert(0,'/home/georgeos/Documents/GitHub/SYMPTOMM2')
import itertools
from joblib import Parallel, delayed
from skimage.morphology import opening
from PIL import Image       
import pymunk
from skimage.transform import PiecewiseAffineTransform, warp
from skimage import data
from copy import deepcopy
from tqdm import tqdm
import pandas as pd
from skimage import draw
#import napari
from itertools import combinations
from SYMPTOMM import PSF
from matplotlib_scalebar.scalebar import ScaleBar
from cupyx.scipy.ndimage import convolve as cuconvolve
import tifffile
from skimage.exposure import match_histograms
import cupy as cp
from scipy.optimize import dual_annealing, shgo
from skimage.transform import resize
from skimage.metrics import structural_similarity as ssim
from scipy.optimize import basinhopping
import image_similarity_measures
from image_similarity_measures.quality_metrics import rmse, psnr, fsim, issm, sre, sam, uiq
from SYMPTOMM.general_drawing import *
from SYMPTOMM.phase_contrast_drawing import *

In [2]:
with open("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/output_pickles/cell_timeseries_short_trench.p", "rb") as f:
    cell_timeseries = pickle.load(f)
with open("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/output_pickles/space_timeseries_short_trench.p", "rb") as f:
    space = pickle.load(f)


In [5]:
main_segments = get_trench_segments(space)

In [8]:
ID_props = generate_curve_props(cell_timeseries)
cell_timeseries_properties = Parallel(n_jobs=14)(delayed(gen_cell_props_for_draw)(a, ID_props) for a in cell_timeseries)
do_transformation = False
scenes = Parallel(n_jobs=14)(delayed(draw_scene)(cell_properties, False) for cell_properties in tqdm(cell_timeseries_properties))

100%|██████████| 399/399 [03:02<00:00,  2.19it/s]


In [92]:
trench_multiplier = 30
cell_multiplier = 2
background_multiplier = 10
offset = 50
generate_PC_OPL(main_segments, offset, scenes[0][0], trench_multiplier,cell_multiplier,background_multiplier)

array([[30., 30., 30., ..., 30., 30., 30.],
       [30., 30., 30., ..., 30., 30., 30.],
       [30., 30., 30., ..., 30., 30., 30.],
       ...,
       [30., 30., 30., ..., 30., 30., 30.],
       [30., 30., 30., ..., 30., 30., 30.],
       [30., 30., 30., ..., 30., 30., 30.]])

In [ ]:
condensers = get_condensers()

W, R, diameter = condensers["Ph2"]
scale = 0.108379937 / 3 #0.35 #micron per pixel
min_sigma = 0.42*0.6/2 / scale # micron
kernel= get_phase_contrast_kernel(R, W, 50, scale, 5, 15)
scalebar = ScaleBar(scale, 'um')
plt.figure(figsize=(5.1,5.1))
plt.gca().add_artist(scalebar)
plt.imshow(kernel,cmap="Greys_r")
plt.axis("off")
plt.title("Phase Contrast")
plt.show()
kernel = cp.array(kernel)
kernel_np = kernel.get()
plt.figure(figsize = (15,10))
plt.plot(np.arange(-50,51)*scale,kernel_np[50,:])
plt.show()